# Common Variables and Imports

In [14]:
import os, time, warnings, pickle
import numpy as np
from sklearn.metrics import accuracy_score, log_loss, pairwise_distances
from sklearn.exceptions import ConvergenceWarning
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling, entropy_sampling, margin_sampling, classifier_uncertainty, classifier_entropy, classifier_margin
from modAL.batch import uncertainty_batch_sampling
from modAL.expected_error import expected_error_reduction
from modAL.density import information_density

from utils import load_MNIST, log_metrics, initialize_random_number_generators, create_model, save_model_and_metrics, load_model_and_metrics

# Filter FutureWarnings to make outputs look more pleasant and ConvergenceWarnings which are given by sklearn LogisticRegressors when explicitly settings the multi_class to multinomial. Here, this could be omitted but I liked to leave it in for clarity to show that I'm not training 10 binary classifiers but one classifier with 10 outputs, each resembling the probabilities of a digit
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [15]:
RANDOM_SEED = 42
epochs = 100 
model_parameters={
    "max_iterations_per_epoch": 10,
    "regularization": 'l2', # l1', 'l2', 'elasticnet' or None
    "regularization_strength": 1.0 # smaller values mean stronger regularization
}

initialize_random_number_generators(RANDOM_SEED)

experiment = "1" # "2" or "3"

In [16]:
initialize_random_number_generators(seed=RANDOM_SEED)

# n_queries = 100 # I set this to have a comparable amount of overall train epohcs on both models
n_query_instances = 5
n_initial = 100

# Load DataSet (MNIST)

In [17]:
X_train, y_train, X_test, y_test, X_whole, y_whole = load_MNIST()

Check how often each digit appears in MNIST and whether the data for each class is balanced

In [18]:
for i in range(10):
    print(f"Digit {i}: {np.count_nonzero(y_whole == i)} times")

Digit 0: 6903 times
Digit 1: 7877 times
Digit 2: 6990 times
Digit 3: 7141 times
Digit 4: 6824 times
Digit 5: 6313 times
Digit 6: 6876 times
Digit 7: 7293 times
Digit 8: 6825 times
Digit 9: 6958 times


# Helper methods

In [19]:
""" 
    Method for pool-based active learning query strategies. 
"""
def train_active_learner(model_params, query_strat, n_query_instances:int, epochs:int, random_seed:int, pool_idx:list[int], X_initial, y_initial):
    initialize_random_number_generators(seed=random_seed)
    
    X_pool = X_train[pool_idx]
    y_pool = y_train[pool_idx]
    
    log_reg = create_model(model_params, random_seed=RANDOM_SEED)
    
    learner = ActiveLearner(estimator=log_reg, 
                        query_strategy=query_strat, 
                        X_training=X_initial, 
                        y_training=y_initial)
    # Passing X_training and y_training to the ActiveLearner automatically calls the fit method for log_reg with these data points

    metrics = {'queries': [], 'train_loss': [], 'train_loss_current': [], 'test_loss': [], 'test_acc': []}
    
    start = time.time()
    for epoch in range(epochs): # epochs=n_queries to have both models trained on the same number of overall epochs
        query_idx, query_inst = learner.query(X_pool, n_instances=n_query_instances)
        
        # Simulate labeling
        learner.teach(X_pool[query_idx], y_pool[query_idx])
        
        # Remove queried point(s)
        X_pool, y_pool = np.delete(X_pool, query_idx, axis=0), np.delete(y_pool, query_idx, axis=0)
    
        metrics['queries'].append(query_idx)
        # log_metrics logs train loss for the whole train dataset which doesn't reflect the actual value in the current step but gives the ability to compare both models on the training set. 
        # To log training state on the actual (current) training set, do this additionally
        train_loss_current = log_loss(learner.y_training, learner.predict_proba(learner.X_training))
        metrics['train_loss_current'].append(train_loss_current)
        
        log_metrics((epoch+1)*model_params["max_iterations_per_epoch"], learner, X_train, y_train, X_test, y_test, metrics)
        print(f"       Current train loss: {train_loss_current:.4f}")
    print(f"Training time: {time.time()-start:.2f} seconds")
    
    return learner.estimator, metrics

""" 
    Method for stream-based active learning query strategies. 
    
    arguments:
    - model_params: dict
    - query_strat: dict
    - query_score_threshold: float
    - epochs: int
    - random_seed: int
    - X_stream: np.ndarray, typically full dataset
    - y_stream: np.ndarray, typically full dataset
    - X_initial: np.ndarray, initial training points
    - y_initial: np.ndarray, initial training points
"""
def train_active_learner_stream(model_params, query_score_fn, n_query_instances:int, query_score_threshold:float, epochs:int, random_seed:int, X_stream, y_stream, X_initial, y_initial):    
    initialize_random_number_generators(seed=random_seed)
    
    log_reg = create_model(model_params, random_seed=RANDOM_SEED)
    
    learner = ActiveLearner(estimator=log_reg, 
                            X_training=X_initial, 
                            y_training=y_initial)
    # Passing X_training and y_training to the ActiveLearner automatically calls the fit method for log_reg with these data points

    metrics = {'queries': [], 'train_loss': [], 'train_loss_current': [], 'test_loss': [], 'test_acc': []}
    
    start = time.time()
    
    # In pool based training, I get n_query_instances in each epoch. To have a comparable amount of data points for retraining the classifier, I use max_instances which equates to the amount of instances, a model in pool based approaches has seen. 
    max_instances = n_query_instances * epochs 
    used_instances = 0
    
    # Use a random permutation of the whole MNIST train data set to mimic a data stream. 
    stream_indices, stream_pointer = np.random.permutation(len(X_stream)), 0
    
    while used_instances < max_instances:
        if stream_pointer >= len(stream_indices):  # Restart stream simulation if exhausted
            stream_indices = np.random.permutation(len(X_stream))
            stream_pointer = 0
        
        stream_idx = stream_indices[stream_pointer]
        x_instance, y_instance = X_stream[stream_idx].reshape(1, -1), y_stream[stream_idx].reshape(-1,)
        stream_pointer += 1
        
        query_score = query_score_fn(learner, x_instance)
        
        # Depending on the function, we want to select samples with either a high score (uncertainty) or a low score (margin)
        
        if query_score_fn == classifier_margin:
            query_condition = query_score < query_score_threshold
        else: # classifier_uncertainty, classifier_entropy
            query_condition = query_score > query_score_threshold
        
        if query_condition:
            learner.teach(x_instance, y_instance)
    
            metrics['queries'].append(stream_idx)
            # log_metrics logs train loss for the whole train dataset which doesn't reflect the actual value in the current step but gives the ability to compare both models on the training set. 
            # To log training state on the actual (current) training set, do this additionally
            train_loss_current = log_loss(learner.y_training, learner.predict_proba(learner.X_training))
            metrics['train_loss_current'].append(train_loss_current)
            
            log_metrics((used_instances+1), learner, X_train, y_train, X_test, y_test, metrics)
            print(f"       Current train loss: {train_loss_current:.4f}")
            
            used_instances += 1
            
    print(f"Training time: {time.time()-start:.2f} seconds")
    
    return learner.estimator, metrics

## Creating an initial labelled dataset from random datapoints and the unlabelled pool

In [20]:
initial_idx = np.random.choice(range(len(X_train)), size=n_initial, replace=False) # Indices with which the initial train set is created with
X_initial = X_train[initial_idx]
y_initial = y_train[initial_idx]
pool_idx = np.setdiff1d(range(len(X_train)), initial_idx)

# Train a Logistic Regressor on the whole train dataset
Use multi_class='multinomial' solver: if it were 'ovr', each of the 10 output neurons would treat the corresponding number as a one-vs-rest szenario. So we would construct a Binary Distribution for each of the output neurons. But this doesn't take care of interdependencies between classes. 
In General: 'ovr' would train a separate classifier for each number and 'multinomial' does a softmax regression

I could use ovr to have more control about the individual classes and to save computational costs 

Other option would be lbfgs which is also compatible with L2 and multiclass

In [ ]:
def train_full_model():
    log_reg_full = create_model(model_parameters, random_seed=RANDOM_SEED)
    
    metrics = {'train_loss': [], 'test_loss': [], 'test_acc': []}
    
    start = time.time()
    for epoch in range(epochs):
        log_reg_full.fit(X_train, y_train)
        log_metrics((epoch+1) * model_parameters["max_iterations_per_epoch"], log_reg_full, X_train, y_train, X_test, y_test, metrics)

    y_hat = log_reg_full.predict(X_test)
    accuracy_whole_dataset = accuracy_score(y_test, y_hat)
    print(f"Test accuracy with whole Test dataset: {accuracy_whole_dataset:.4f}")
    print(f"Training time: {time.time() - start:.2f} seconds")
    
    return log_reg_full, metrics

log_reg_whole_data, log_reg_whole_data_metrics = train_full_model()
save_model_and_metrics(experiment, "whole_dataset", log_reg_whole_data, log_reg_whole_data_metrics)

Iteration 10: - Train Loss: 0.2365 - Test Loss: 0.2663 - Test Accuracy: 0.9259


---

# Active Learning

---

### Train a Logistic Regressor on 100 data points without Active Learning

This serves as a baseline to show how much can be learnt from n_initial data points. (This is expected to be low)

In [ ]:
def train_initial_model():
    initialize_random_number_generators(seed=RANDOM_SEED)
    
    log_reg_initial = create_model(model_parameters, random_seed=RANDOM_SEED)
    
    metrics = {'train_loss': [], 'train_loss_current': [], 'test_loss': [], 'test_acc': []}
    
    start = time.time()
    log_reg_initial.fit(X_initial, y_initial)
    print(f"Train time: {time.time() - start:.2f} seconds")
    
    y_pred = log_reg_initial.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
     
    train_loss_current = log_loss(y_initial, log_reg_initial.predict_proba(X_initial))
    metrics['train_loss_current'].append(train_loss_current)
    
    log_metrics(model_parameters["max_iterations_per_epoch"], log_reg_initial, X_train, y_train, X_test, y_test, metrics)

    print(f"Test accuracy with whole Test dataset: {accuracy:.4f}")
    
    return log_reg_initial, metrics

log_reg_initial, log_reg_initial_metrics = train_initial_model()
save_model_and_metrics(experiment, "initial_active_model", log_reg_initial, log_reg_initial_metrics)

## Train a Classifier with Various Query Strategies

From the documents and maybe worth trying: If you would like to start from scratch, you can use the .fit(X, y) method to make the learner forget everything it has seen and fit the model to the newly provided data.

To train only on the newly acquired data, you should pass only_new=True to the .teach() method. 

## Random Sampling

In [ ]:
def random_sampling(classifier, X_pool, n_instances):
    n_samples = len(X_pool)
    query_idx = np.random.choice(range(n_samples), size=n_instances, replace=False)
    return query_idx, X_pool[query_idx]

learner, metrics = train_active_learner(model_params=model_parameters, query_strat=random_sampling, n_query_instances=n_query_instances, epochs=epochs, random_seed=RANDOM_SEED, pool_idx=pool_idx, X_initial=X_initial, y_initial=y_initial)
save_model_and_metrics(experiment, "random_sampling", learner, metrics)

## Uncertainty sampling strategies

**Uncertainty Sampling**: Samples where classifier is least sure are selected

In [ ]:
learner, metrics = train_active_learner(model_params=model_parameters, query_strat=uncertainty_sampling, n_query_instances=n_query_instances, epochs=epochs, random_seed=RANDOM_SEED, pool_idx=pool_idx, X_initial=X_initial, y_initial=y_initial)
save_model_and_metrics(experiment, "uncertainty_sampling", learner, metrics)

**Entropy Sampling**: Samples where class probability has the largest Entropy

In [ ]:
learner, metrics = train_active_learner(model_params=model_parameters, query_strat=entropy_sampling, n_query_instances=n_query_instances, epochs=epochs, random_seed=RANDOM_SEED, pool_idx=pool_idx, X_initial=X_initial, y_initial=y_initial)
save_model_and_metrics(experiment, "uncertainty_entropy_sampling", learner, metrics)

**Margin Sampling**: Selects instances where difference between first most likely and second most likely classes are the smallest

In [ ]:
learner, metrics = train_active_learner(model_params=model_parameters, query_strat=margin_sampling, n_query_instances=n_query_instances, epochs=epochs, random_seed=RANDOM_SEED, pool_idx=pool_idx, X_initial=X_initial, y_initial=y_initial)
save_model_and_metrics(experiment, "uncertainty_margin_sampling", learner, metrics)

## Ranked Batch-Mode Sampling

$$score=\alpha\left(1-\Phi\left(x, X_{\text {labeled }}\right)\right)+(1-\alpha) U(x)$$
where $\alpha=\frac{\left|X_{\text {unlabeled }}\right|}{\left|X_{\text {unlabeled }}\right|+\left|X_{\text {labeled }}\right|}, X_{\text {labeled }}$ is the labeled dataset, $U(x)$ is the uncertainty of predictions for $x$, and $\Phi$ is a so-called similarity function, for instance cosine similarity. This latter function measures how well the feature space is explored near $x$. (The lower the better.)

According to the modAL docs: This strategy differs from uncertainty_sampling() because, although it is supported, traditional active learning query strategies suffer from sub-optimal record selection when passing n_instances > 1. This sampling strategy extends the interactive uncertainty query sampling by allowing for batch-mode uncertainty query sampling. Furthermore, it also enforces a ranking – that is, which records among the batch are most important for labeling?

In [ ]:
learner, metrics = train_active_learner(model_params=model_parameters, query_strat=uncertainty_batch_sampling, n_query_instances=n_query_instances, epochs=epochs, random_seed=RANDOM_SEED, pool_idx=pool_idx, X_initial=X_initial, y_initial=y_initial)
save_model_and_metrics(experiment, "ranked_batch_mode", learner, metrics)

## Custom combination between uncertainty of the data point(s) in X and diversity between X and parts of the train dataset

In [ ]:
alpha_uc_dv = 0.5
def ranked_uc_and_dv_score(learner, X):
    uncertainty = classifier_uncertainty(learner, X)
    diversity = np.min(pairwise_distances(X, learner.X_training), axis=1)
    combined_scores = alpha_uc_dv * uncertainty + (1 - alpha_uc_dv) * diversity
    return combined_scores

def ranked_uc_and_dv_query(learner, X, n_instances=1):
    uc_dv_scores = ranked_uc_and_dv_score(learner, X)
    # Sort them in descending order
    ranked_indices = np.argsort(uc_dv_scores)[::-1]
    selected_indices = ranked_indices[:n_instances]
    selected_instances = X[selected_indices]
    
    return selected_indices, selected_instances

for a, a_str in [(0.2, "0_2"), (0.5, "0_5"), (0.8, "0_8")]:
    alpha_uc_dv = a
    learner, metrics = train_active_learner(model_params=model_parameters, query_strat=ranked_uc_and_dv_query, n_query_instances=n_query_instances, epochs=epochs, random_seed=RANDOM_SEED, pool_idx=pool_idx, X_initial=X_initial, y_initial=y_initial)
    save_model_and_metrics(experiment, f"ranked_uc_and_dv_{a_str}", learner, metrics)

## Stream-Based sampling

with Uncertainty Sampling/ Classifier uncertainty as it's query score method

In [ ]:
for uncertainty_threshold, uncertainty_threshold_str in [(0.2, "0_2"), (0.5, "0_5"), (0.8, "0_8")]:
    learner, metrics = train_active_learner_stream(model_params=model_parameters, query_score_fn=classifier_uncertainty, query_score_threshold=uncertainty_threshold, n_query_instances=n_query_instances, epochs=epochs, random_seed=RANDOM_SEED, X_stream=X_train, y_stream=y_train, X_initial=X_initial, y_initial=y_initial)
    save_model_and_metrics(experiment, f"stream_classifier_uncertainty_th_{uncertainty_threshold_str}", learner, metrics)

with Classification margin uncertainty as it's query score method

In [ ]:
learner, metrics = train_active_learner_stream(model_params=model_parameters, query_score_fn=classifier_margin, query_score_threshold=0.5, n_query_instances=n_query_instances, epochs=epochs, random_seed=RANDOM_SEED, X_stream=X_train, y_stream=y_train, X_initial=X_initial, y_initial=y_initial)
save_model_and_metrics(experiment, "stream_classifier_margin", learner, metrics)

with Entropy margin uncertainty as it's query score method

In [ ]:
learner, metrics = train_active_learner_stream(model_params=model_parameters, query_score_fn=classifier_entropy, query_score_threshold=0.5, n_query_instances=n_query_instances, epochs=epochs, random_seed=RANDOM_SEED, X_stream=X_train, y_stream=y_train, X_initial=X_initial, y_initial=y_initial)
save_model_and_metrics(experiment, "stream_entropy", learner, metrics)

with the custom measurement of uncertainty and diversity of the already seen datapoints

In [ ]:
for uncertainty_threshold, uncertainty_threshold_str in [(0.2, "0_2"), (0.5, "0_5"), (0.8, "0_8")]:
    for a, a_str in [(0.2, "0_2"), (0.5, "0_5"), (0.8, "0_8")]:
        alpha_uc_dv = a
        learner, metrics = train_active_learner_stream(
            model_params=model_parameters, 
            query_score_fn=ranked_uc_and_dv_score, 
            query_score_threshold=uncertainty_threshold, 
            n_query_instances=n_query_instances, 
            epochs=epochs, random_seed=RANDOM_SEED, 
            X_stream=X_train, 
            y_stream=y_train, 
            X_initial=X_initial, 
            y_initial=y_initial)
        save_model_and_metrics(experiment, 
                               f"stream_classifier_ranked_uc_and_dv_{a_str}_th_{uncertainty_threshold_str}", 
                               learner, metrics)

## Disagreement Sampling (for classifiers) (uses a committee, so I should theoretically do every train run again for each methodwith a committee!)

---

# Methods that sadly don't work 

---

## Expected error reduction (doesn't work on my pc due to time complexity)

In [ ]:
# learner, metrics = train_active_learner(model_params=model_parameters, query_strat=expected_error_reduction, epochs=epochs, random_seed=RANDOM_SEED, pool_idx=pool_idx, X_initial=X_initial, y_initial=y_initial)

## Information Density (doesn't work on my pc due to time complexity)

$$I(x)=\frac{1}{\left|X_u\right|} \sum_{x^{\prime} \in X} \operatorname{sim}\left(x, x^{\prime}\right)$$

where $\operatorname{sim}\left(x, x^{\prime}\right)$ is a similarity function such as cosine similarity or Euclidean similarity, which is the reciprocal of Euclidean distance. The higher the information density, the more similar the given instance is to the rest of the data.


According to the modAL docs: When using uncertainty sampling (or other similar strategies), we are unable to take the structure of the data into account which can lead to suboptimal queries.

This could very well be used in combination with another strategy

In [ ]:
# def inf_density(classifier, X_pool):
#     return information_density(X_pool, metric='euclidean')
# 
# learner, metrics = train_active_learner(model_params=model_parameters, query_strat=inf_density,  epochs=epochs, random_seed=RANDOM_SEED, pool_idx=pool_idx, X_initial=X_initial, y_initial=y_initial)

---

This is for using it with a Committee (for multiple classes) so it's not optimal for comparing the query strategies themselves maybe?

---

Acquisition Functions might not be usable due to the fact that they require a BayesianOptimizer, not an ActiveLearner

## Acquisition Functions

**Probability of improvement**: 
$$PI(x)=\psi\left(\frac{\mu(x) - f\left(x^+\right) - \xi}{\sigma(x)}\right)$$
where $\mu(x)$ and $\sigma(x)$ are mean and variance of regressor at $x$, $f$ is the model to be optimized with estimated maximum at $x^+$. $\xi$ is a parameter controlling the degree of exploration and $\psi(x)$ denotes cumulative distribution function of a standard Gaussian Distribution

[Example from the ModAL Docs](https://modal-python.readthedocs.io/en/latest/_images/bo-PI.png)


In [ ]:
# learner, metrics = train_active_learner(model_params=model_parameters, query_strat=max_PI, epochs=epochs, random_seed=RANDOM_SEED, pool_idx=pool_idx, X_initial=X_initial, y_initial=y_initial)

**[Expected Improvement (from the ModAL Docs)](https://modal-python.readthedocs.io/en/latest/content/query_strategies/Acquisition-functions.html#expected-improvement)**: 
$$
EI(x) = 
\left( \mu(x) - f(x^+) - \xi \right) \cdot \psi \left( \frac{\mu(x) - f(x^+) - \xi}{\sigma(x)} \right)
+ \sigma(x) \phi \left( \frac{\mu(x) - f(x^+) - \xi}{\sigma(x)} \right),
$$

where $\mu(x)$ and $\sigma(x)$ are the mean and variance of the regressor at $x$, $f$ is the function to be optimized with estimated maximum at $x$, $\xi$ is a parameter controlling the degree of exploration, and $\psi(z), \phi(z)$ denote the cumulative distribution function and density function of a standard Gaussian distribution.

**Upper Confidence Bound**:
$$ UCB(x) = \mu(x) + \beta \sigma(x)$$
where $\mu(x)$ and $\sigma(x)$ are mean and variance of the regressor and $\beta$ is a parameter controlling the degree of exploration

[Example from the ModAL Docs](https://modal-python.readthedocs.io/en/latest/_images/bo-UCB.png)